In [ ]:
from google.colab import drive
drive.mount('/content/data')

Mounted at /content/data


In [ ]:
import numpy as np
import os
import cv2
import random
from sklearn.preprocessing import MultiLabelBinarizer

def load_dataset(data_dir, num_classes, test=False):
    images = []
    labels = []

    # Paths to images and their corresponding label files
    images_dir = os.path.join(data_dir, "images")
    labels_dir = os.path.join(data_dir, "labels")

    for image_name in os.listdir(images_dir):
        image_path = os.path.join(images_dir, image_name)
        # Preprocess the original image
        #read the original image
        original_image = cv2.imread(image_path)
        #resize the original image
        original_image = cv2.resize(original_image, (224, 224))
        images.append(original_image)

        # Apply rotation and translation augmentation (only for training data)
        if not test:
            # Randomly rotate the image by -15 to +15 degrees
            angle = random.randint(-15, 15)
            rows, cols, _ = original_image.shape
            rotation_matrix = cv2.getRotationMatrix2D((cols / 2, rows / 2), angle, 1)
            rotated_image = cv2.warpAffine(original_image, rotation_matrix, (cols, rows))

            # Randomly translate the image by -10 to +10 pixels horizontally and vertically
            tx = random.randint(-10, 10)
            ty = random.randint(-10, 10)
            translation_matrix = np.float32([[1, 0, tx], [0, 1, ty]])
            translated_image = cv2.warpAffine(rotated_image, translation_matrix, (cols, rows))

            # Append rotated and translated images
            images.append(rotated_image)
            images.append(translated_image)

        # Read labels from the corresponding label file if it exists
        label_file_path = os.path.join(labels_dir, image_name.replace(".jpg", ".txt"))
        if os.path.exists(label_file_path):
            label_vector = []  # Initialize label vector
            with open(label_file_path, 'r') as label_file:
                # There can be multiple labels per image
                for line in label_file:
                    # Extract class index from the line
                    class_index = int(line.split()[0])
                    label_vector.append(class_index)
            labels.append(label_vector)
            # Duplicate labels for rotated and translated images (only for training data)
            if not test:
                labels.append(label_vector)
                labels.append(label_vector)
        else:
            # If label file is missing, mark the label as an empty list
            labels.append([])
            # Duplicate labels for rotated and translated images (only for training data)
            if not test:
                labels.append([])
                labels.append([])

    # Convert labels to binary format using MultiLabelBinarizer
    mlb = MultiLabelBinarizer(classes=range(num_classes))
    labels = mlb.fit_transform(labels)

    return np.array(images), labels

# Specify the path to your dataset directories and the number of classes
train_data_dir = "/content/data/My Drive/AI_Project/train"
test_data_dir = "/content/data/My Drive/AI_Project/test"
num_classes = 10

# Load and preprocess the training dataset
train_images, train_labels = load_dataset(train_data_dir, num_classes)
print("Shape of training images array:", train_images.shape)
print("Shape of training labels array:", train_labels.shape)

# Load and preprocess the test dataset
test_images, test_labels = load_dataset(test_data_dir, num_classes, test=True)
print("Shape of test images array:", test_images.shape)
print("Shape of test labels array:", test_labels.shape)


Shape of training images array: (3036, 224, 224, 3)
Shape of training labels array: (3036, 10)
Shape of test images array: (107, 224, 224, 3)
Shape of test labels array: (107, 10)


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import precision_score
import numpy as np

X_train = train_images.reshape(train_images.shape[0], -1)  # Flatten images
y_train = train_labels

X_test = test_images.reshape(test_images.shape[0], -1)
y_test = test_labels

# Create a KNeighborsClassifier for each label
knn_classifiers = [KNeighborsClassifier(n_neighbors=11, algorithm='auto', metric='euclidean') for _ in range(y_train.shape[1])]

# Train each classifier on the corresponding label
for i, classifier in enumerate(knn_classifiers):
    classifier.fit(X_train, y_train[:, i])

# Calculate accuracy for each label
accuracies = [classifier.score(X_test, y_test[:, i]) for i, classifier in enumerate(knn_classifiers)]

# Calculate overall accuracy with weighted average
weighted_accuracies = [accuracy * len(y_test[:, i]) / len(y_test) for i, accuracy in enumerate(accuracies)]
overall_accuracy = np.mean(accuracies)
print("Overall Accuracy:", overall_accuracy)

# Initialize lists to store evaluation results
precisions = []


# Evaluate each classifier
for i, classifier in enumerate(knn_classifiers):
    # Predict binary labels
    y_pred = classifier.predict(X_test)
    # Calculate precision
    precision = precision_score(y_test[:, i], y_pred, zero_division=1)
    precisions.append(precision)


# Calculate overall metrics
overall_precision = np.mean(precisions)


print("Overall Precision:", overall_precision)



Overall Accuracy: 0.9093457943925234
Overall Precision: 0.5
